In [81]:
# # install AMPL
# # Install Python API for AMPL:
# $ python -m pip install amplpy --upgrade

# # Install solver modules:
# $ python -m amplpy.modules install highs

# # Activate your AMPL CE license: ............................. (paste id license)
# $ python -m amplpy.modules activate

In [85]:
from amplpy import AMPL, tools
# ampl = tools.ampl_notebook(
#     modules=["highs", "cbc", "gurobi"], # modules to install
#     license_uuid="d3af9008-221f-4220-a118-625786b1fe84")



Licensed to AMPL Community Edition License for <lorenzo.scaioli0@gmail.com>.


# Model

In [87]:
model = r"""
    reset;

    ## VARIABLES
    param m;
    param n;
    set COURIERS := {1..m}; # couriers with load capacities
    set ITEMS := {1..n}; # items with sizes
    set D_SIZE := {1..n+1};

    param capacity {COURIERS} > 0 integer;
    param size {ITEMS} > 0 integer;
    param D {D_SIZE, D_SIZE} >= 0 integer; # matrix of distances
    param dist_upper_bound := sum {i in D_SIZE} max {j in D_SIZE} D[i,j];
    param obj_lower_bound := max {i in ITEMS} (D[n+1,i]+D[i,n+1]);


    var X {COURIERS, D_SIZE, D_SIZE} binary; # tensor defining the route of each courier
    var T {ITEMS} >= 1, <= n integer; # array that encode the visit sequence
    # var items_per_courier {COURIERS} integer;
    # TODO: if implicit constraint, add lower_bound also on tot_dist
    var tot_dist {COURIERS} >= 0, <= dist_upper_bound integer; # distance traveled by each courier
    var Obj >= obj_lower_bound, <= dist_upper_bound integer;

    ## OBJECTIVE FUNCTION
    minimize Obj_function: Obj;

    ## CONSTRAINTS
    ## constraints on Obj
    s.t. def_Obj {i in COURIERS}:
        Obj >= tot_dist[i];
     
    ## constraints to create X 
    s.t. X_1 {k in ITEMS}:
        sum {i in COURIERS, j in D_SIZE} X[i,j,k] = 1; # each X[:,:,k] matrix has exaclty 1 item, just one i courier arrive at k-th point
    s.t. X_2 {j in ITEMS}:
        sum {i in COURIERS, k in D_SIZE} X[i,j,k] = 1; # each X[:,j,:] matrix has exaclty 1 item, just one i courier depart from j-th point
    s.t. X_3 {i in COURIERS}:
        sum {j in ITEMS} X[i,j,n+1] = 1; # each X[i,:,n+1] column has exactly 1 item, the courier i return at the origin
    s.t. X_4 {i in COURIERS}:
        sum {k in ITEMS} X[i,n+1,k] = 1; # each X[i,n+1,:] row has exactly 1 item, the courier i start from the origin
    s.t. X_5 {i in COURIERS, j in ITEMS}:
        X[i,j,j] = 0; # the diagonal of each X[i,:,:] is zero, the i courier must move from a point to another
    s.t. implied_constraint {i in COURIERS}: # TODO
        X[i,n+1,n+1] = 0; # each courier transoprts at least one item
    s.t. X_6 {i in COURIERS, j in ITEMS}:
        sum {k in D_SIZE} X[i,k,j] = sum {k in D_SIZE} X[i,j,k]; # for each i courier the sum of each column A[i,:,j] is equal to the sum of each row A[i,j,:]
                                                                 # if the i courier enter arrive at the j-th point it has to depart from it
    s.t. load_capacity {i in COURIERS}:
        sum {j in D_SIZE, k in ITEMS} X[i,j,k]*size[k] <= capacity[i]; # each courier respects its own load capacity 

    ## constraints to create T
    s.t. T_1 {i in COURIERS, j in ITEMS, k in ITEMS}:
        T[j]-T[k] >= 1 - 2*n * (1-X[i,k,j]); # if the X[i,j,k] is 1 (vehicle i leaves node k and enter the node j) then T[j]-T[i]=1, the point j-th is visited exactly after the k-th point
                                             # value of big-M = 2*n
    s.t. T_2 {i in COURIERS, j in ITEMS, k in ITEMS}:
        T[j]-T[k] <= 1 + 2*n * (1-X[i,k,j]);
    s.t. T_3 {i in COURIERS, k in ITEMS}:
        T[k] <= 1 + 2*n * (1-X[i,n+1,k]); # for every courier the first element delivered, call it k, gets T[k]=1
          
    ## constraint to create tot_dist[i]
    s.t. D_1 {i in COURIERS}:
        sum {j in D_SIZE, k in D_SIZE} X[i,j,k] * D[j,k] = tot_dist[i]; # calculate distance traveled by each courier
          
    ## symmetry breaking with ordered capacity 
    s.t. symmetry_breaking {i in {1..m-1}}:
        sum {j in ITEMS, k in ITEMS} X[i,j,k]*size[k] >= sum {j in ITEMS, k in ITEMS} X[i+1,j,k]*size[k]; # the load of each courier is ordered as the capacity
          
"""


## Solve with COIN-BC

In [88]:
# # Specify the solver to use (e.g., HiGHS)
# ampl.option["solver"] = "cbc"
# # Solve
# ampl.solve()
# # Stop if the model was not solved
# assert ampl.get_value("solve_result") == "solved"
# # Get objective entity by AMPL name
# totalcost = ampl.get_objective('Obj_function')
# # Print it
# print("Objective is:", int(round(totalcost.value(),0)))

# Testing

In [92]:
def run_model(model, m, n, l, s, D, **kwargs):
    # TODO: if symmetry breaking:
    l.sort(reverse=True)

    ampl = AMPL()
    ampl.eval(model)

    ampl.param["m"] = m
    ampl.param["n"] = n

    ampl.param["capacity"] = l
    ampl.param["size"] = s
    ampl.param["D"] = D

    # Specify the solver to use (e.g., HiGHS)
    ampl.option["solver"] = 'gurobi'
    ampl.option["gurobi_options"] = "timelim=300"  # TODO: deve diventare f'{solver}_options
    # Solve
    ampl.solve()
    # Stop if the model was not solved
    solve_result = ampl.get_value("solve_result")

    # Get objective entity by AMPL name
    obj_value = int(round(ampl.get_objective('Obj_function').value(), 0))

    if solve_result == "solved":
        print(f"SAT -- obj_value: {obj_value}")
    elif solve_result == "limit" and obj_value == 0:
        print('Timeout, no solution found') 
    else:
        print(f"Timeout but found solution with obj_value:{obj_value}")
    

In [93]:
import os
import sys
import numpy as np
from amplpy import AMPL

def run_model_on_instance(MCP_model, file, **kwargs):
    """Read the instance from .dat file and run the given MCP model on it

    Args:
        MCP_model (function): function executing the SAT-encoding and solving of the given instance
        file (str): path of the .dat file representing the instance
    """
    with open(file) as f:
        m = int(next(f))
        n = int(next(f))
        l = [int(e) for e in next(f).split()]
        s = [int(e) for e in next(f).split()]
        D = np.ravel(np.genfromtxt(f, dtype=int)).tolist()

    return run_model(MCP_model, m, n, l, s, D, **kwargs)

In [94]:
run_model_on_instance(model, '../instances/inst07.dat') 

Gurobi 10.0.1: lim:time=300
Gurobi 10.0.1: optimal solution; objective 167
139310 simplex iterations
2780 branching nodes
SAT -- obj_value: 167
